In [ ]:
# Import a 12-channel ECG recording and visualize the recordings

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

'''
# This sets up the matplotlib interactive windows:
#!pip install ipympl
import ipympl
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()
'''

'''
# Clone the data files
!git clone https://github.com/BMEn-datascience/Lecture02.git

# Let's import the .csv files
data = pd.read_csv('Lecture02/ecg_incartdb_I04.csv')
'''

# Or, you can load the DataFrame from a CSV file
df = pd.read_csv('ecg_incartdb_I04.csv')

# Define the sampling rate
fs = 257 # Hz (4 ms between sample)

# Create a column of time stamps
t = np.arange(0,len(df))/fs
df.insert(0,'Time',t)

# Slice data to zoom in on just a few beats (first 5 seconds in this case)
tstamp = 5 # sec
df_slice = df.iloc[:tstamp*fs]
print(df_slice)

# If you want to access the first row of data
first_row = df_slice.iloc[0]
print(first_row)

# Remove the amplitude offset from each recording channel
df_sub = df_slice.subtract(first_row)

# Plot each channel
fig = plt.figure(figsize=(6,6))
for i in range(1, 13):  # Lead I through Lead V6
  plt.plot(df_sub['Time'], df_sub.iloc[:, i]-1*i, label=f'{df_sub.columns[i]}')
plt.xlabel('Time (s)')
plt.ylabel('ECG Amplitude (mV)')
plt.title('12-Channel ECG Recording')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Import a single-channel microelectrode brain recording and visualize the recordings

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Clone the data files
!git clone https://github.com/BMEn-datascience/Lecture02.git

# Let's import the .csv files
lfp_data = pd.read_csv('Lecture02/lfp-data.csv', names=['lfp'])
spike_data = pd.read_csv('Lecture02/spike-data.csv', names=['spike'])

# Define the sampling rate
fs_lfp = 1395.0893    # Hz
fs_spike = 22321.4283 # Hz

# Create a column of time stamps in each DataFrame
t_lfp = np.arange(0,len(lfp_data))/fs_lfp
lfp_data.insert(0,'time',t_lfp)
t_spike = np.arange(0,len(spike_data))/fs_spike
spike_data.insert(0,'time',t_spike)

# Plot the lfp signal
plt.figure(figsize=(6,6))
plt.plot(lfp_data['time'], lfp_data['lfp'], label='LFP')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.title('LFP Filtered Recording')
plt.grid()
plt.show()

# Plot the spectrogram of the lfp
fig = plt.figure(figsize=(6,6))
pxx, freq, t, ax = plt.specgram(lfp_data['lfp'], Fs=fs_lfp, cmap='jet', NFFT=1024, scale='dB')
fig.colorbar(ax).set_label('Power (dB)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('LFP Spectrogram')
plt.grid()
#plt.ylim(0,100)  # Set the y-axis limits to 0-100 Hz
plt.show()

# Filter the lfp and plot the spectrogram
from scipy import signal
b, a = signal.butter(3, 200/(fs_lfp/2), btype='low', analog=True)
lfp_data['lfp_filt'] = signal.filtfilt(b, a, lfp_data['lfp'])
plt.figure(figsize=(6,6))
plt.subplot(2,1,1)
plt.specgram(lfp_data['lfp'], Fs=fs_lfp, cmap='jet', NFFT=1024)
plt.subplot(2,1,2)
plt.specgram(lfp_data['lfp_filt'], Fs=fs_lfp, cmap='jet', NFFT=1024)
plt.show()

# Plot the spike data
plt.figure(figsize=(6,6))
plt.plot(spike_data['time'], spike_data['spike'], label='Spike')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.title('Spike Filtered Recording')
plt.grid()

# Identify spike timestamps within a spike recording and plot them
import scipy.signal as signal
peaks, _ = signal.find_peaks(spike_data['spike'], height=30)
plt.plot(peaks/fs_spike,spike_data['spike'].iloc[peaks], 'x', color='r', label='Peaks')
plt.show()